In [1]:
import pandas as pd
import numpy as np

In [2]:
permits = pd.read_csv('../data/Residential_Short_Term_Rental_Permits.csv')
permits.head()

,Permit #,Applicant,Contact,Permit Subtype Description,Permit Status,Parcel,Date Entered,Date Issued,Expiration Date,Address,...,Permit Owner Addr. 2,Permit Owner Addr. 3,Permit Owner City,Permit Owner State,Permit Owner ZIP,Permit Type,Permit Subtype,Council District,Census Tract,Mapped Location
0,201622784,"Market Street Emporium, Llc",Market Street,Short Term Rental � Multifamily,EXPIRED,9306208400,5/16/2016 0:00,6/16/2016 0:00,6/16/2022 0:00,112 2ND AVE N 102,...,112 2ND AVE N 3RD FL,NaN,NASHVILLE,TN,37201,CASR,CAZ10A003,19.0,37019500,"112 2ND AVE N 102\nNASHVILLE, TN 37201\n(36.16..."
1,201530645,"Young, Christopher A. & Daniella C.","Young, Christopher A. & Daniella C.",Short Term Rental - Owner Occupied,EXPIRED,7310000200,7/21/2015 0:00,7/21/2015 0:00,7/21/2022 0:00,2610 SHADOW LN,...,2610 SHADOW LN,NaN,NASHVILLE,TN,37216,CASR,CAZ10A001,7.0,37011500,"2610 SHADOW LN\nNASHVILLE, TN 37216\n(36.20303..."
2,201526852,"Sinclair, Constance A.","Sinclair, Constance A.",Short Term Rental - Owner Occupied,ISSUED,9402015700,6/26/2015 0:00,6/26/2015 0:00,6/26/2023 0:00,1909 LONG AVE,...,1909 LONG AVE,NaN,NASHVILLE,TN,37203,CASR,CAZ10A001,6.0,37012200,"1909 LONG AVE\nNASHVILLE, TN 37206\n(36.168897..."
3,201622787,"Market Street Emporium, Llc",Market Street,Short Term Rental � Multifamily,EXPIRED,9306208400,5/16/2016 0:00,6/16/2016 0:00,6/16/2022 0:00,112 2ND AVE N 104,...,112 2ND AVE N 3RD FL,NaN,NASHVILLE,TN,37201,CASR,CAZ10A003,19.0,37019500,"112 2ND AVE N 104\nNASHVILLE, TN 37201\n(36.16..."
4,201532188,"Patterson, Travis & Michelle & Stephens, Gary",Michelle Patterson,Short Term Rental - Non-Owner Occupied,ISSUED,8313053700,7/30/2015 0:00,7/30/2015 0:00,7/30/2023 0:00,1105 LILLIAN ST,...,1105 LILLIAN ST,NaN,NASHVILLE,TN,37206,CASR,CAZ10A002,6.0,37019200,"1105 LILLIAN ST\nNASHVILLE, TN 37206\n(36.1733..."


In [3]:
permits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13279 entries, 0 to 13278
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Permit #                    13279 non-null  object 
 1   Applicant                   13273 non-null  object 
 2   Contact                     12888 non-null  object 
 3   Permit Subtype Description  13279 non-null  object 
 4   Permit Status               13279 non-null  object 
 5   Parcel                      13279 non-null  object 
 6   Date Entered                13279 non-null  object 
 7   Date Issued                 11870 non-null  object 
 8   Expiration Date             13275 non-null  object 
 9   Address                     13279 non-null  object 
 10  City                        13278 non-null  object 
 11  State                       13278 non-null  object 
 12  ZIP                         13279 non-null  int64  
 13  Subdivision / Lot           132

In [4]:
permits['ZIP'] = permits['ZIP'].apply(str)
permits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13279 entries, 0 to 13278
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Permit #                    13279 non-null  object 
 1   Applicant                   13273 non-null  object 
 2   Contact                     12888 non-null  object 
 3   Permit Subtype Description  13279 non-null  object 
 4   Permit Status               13279 non-null  object 
 5   Parcel                      13279 non-null  object 
 6   Date Entered                13279 non-null  object 
 7   Date Issued                 11870 non-null  object 
 8   Expiration Date             13275 non-null  object 
 9   Address                     13279 non-null  object 
 10  City                        13278 non-null  object 
 11  State                       13278 non-null  object 
 12  ZIP                         13279 non-null  object 
 13  Subdivision / Lot           132

In [5]:
permits.value_counts('Permit Status')

Permit Status
ISSUED       5953
EXPIRED      5806
CANCELLED    1296
PENDING       203
HOLD           11
REVOKED         7
EXPIREEXT       2
REFUNDED        1
dtype: int64

In [6]:
permits.value_counts('Applicant')

Applicant
Niido Nashville, LLC             339
SIXTH SOUTH PARTNERS, LP         169
kenny mccullum                   125
TBR-L VILLAGE 21 OWNER LLC        78
VIBE AT MARATHON VILLAGE, LLC     72
                                ... 
George, Regis III                  1
George Williams                    1
George K.Van Oster                 1
George Floyd                       1
zuker, clark                       1
Length: 8879, dtype: int64

In [7]:
permits.value_counts('Permit Owner ZIP').head()

Permit Owner ZIP
37206    1329
37203     921
37215     692
37216     636
37207     634
dtype: int64

In [8]:
permits.value_counts('Permit Owner City').head()

Permit Owner City
NASHVILLE        7497
Nashville        1456
MIAMI             344
BRENTWOOD         237
San Francisco     165
dtype: int64

In [9]:
permits['Permit Owner City'] = permits['Permit Owner City'].str.lower()

In [10]:
permits['Permit Owner State'] = permits['Permit Owner State'].str.upper()

In [11]:
permit_city = permits.value_counts('Permit Owner City')

In [12]:
permit_city

Permit Owner City
nashville     8958
miami          344
brentwood      313
franklin       216
madison        168
              ... 
islamorada       1
irvine           1
iowa city        1
inglewood        1
zionsville       1
Length: 555, dtype: int64

In [13]:
#permit_city['Permit Owner City'].value_counts(normalize=True) * 100

In [14]:
permit_owners_cities = permit_city.to_frame().reset_index()
permit_owners_cities

,Permit Owner City,0
0,nashville,8958
1,miami,344
2,brentwood,313
3,franklin,216
4,madison,168
...,...,...
550,islamorada,1
551,irvine,1
552,iowa city,1
553,inglewood,1


In [15]:
permits_owner_location = permits[['Applicant', 'Permit Owner City', 'Permit Owner State']]
permit_owners_location = permits_owner_location.value_counts().to_frame().reset_index()

In [16]:
permit_owners_location.columns = ['Applicant', 'Permit Owner City', 'Permit Owner State', 'Count of Permits']

In [17]:
permit_owners_location

,Applicant,Permit Owner City,Permit Owner State,Count of Permits
0,"Niido Nashville, LLC",miami,FL,339
1,"SIXTH SOUTH PARTNERS, LP",san francisco,CA,164
2,kenny mccullum,nashville,TN,125
3,TBR-L VILLAGE 21 OWNER LLC,atlanta,GA,78
4,"VIBE AT MARATHON VILLAGE, LLC",nashville,TN,72
...,...,...,...,...
9065,"Gerbig, Chris",bowling green,KY,1
9066,"Gerber, Saline Brooks & Brown, Matthew",nashville,TN,1
9067,Gerardo Castro,chicago,IL,1
9068,Gerald Castle,nashville,TN,1


In [18]:
permit_owners_location.value_counts('Permit Owner State') 

Permit Owner State
TN    7885
CA     279
TX      96
IL      93
NY      83
GA      54
FL      53
OH      45
KY      41
CO      36
IN      31
AZ      26
MA      26
AL      26
NC      23
MI      22
ON      17
PA      17
MO      16
IA      16
LA      15
WA      15
NJ      13
VA      13
CT      13
UT      11
WI      11
NV      10
MD       8
SC       8
MS       7
WY       6
OR       5
DC       5
ID       4
NE       4
OK       4
AR       4
MN       4
KS       4
UN       3
DE       3
NM       3
SD       3
HI       2
NH       2
MT       2
HR       1
WV       1
AB       1
dtype: int64

#13% are out of TN

In [19]:
permit_owners_location['Permit Owner State'].value_counts(normalize=True) * 100

TN    86.934950
CA     3.076075
TX     1.058434
IL     1.025358
NY     0.915105
GA     0.595369
FL     0.584344
OH     0.496141
KY     0.452040
CO     0.396913
IN     0.341786
AZ     0.286659
AL     0.286659
MA     0.286659
NC     0.253583
MI     0.242558
PA     0.187431
ON     0.187431
IA     0.176406
MO     0.176406
WA     0.165380
LA     0.165380
NJ     0.143330
VA     0.143330
CT     0.143330
WI     0.121279
UT     0.121279
NV     0.110254
MD     0.088203
SC     0.088203
MS     0.077178
WY     0.066152
DC     0.055127
OR     0.055127
NE     0.044101
OK     0.044101
AR     0.044101
ID     0.044101
MN     0.044101
KS     0.044101
SD     0.033076
UN     0.033076
NM     0.033076
DE     0.033076
NH     0.022051
MT     0.022051
HI     0.022051
AB     0.011025
WV     0.011025
HR     0.011025
Name: Permit Owner State, dtype: float64

In [20]:
council_district = permits.value_counts('Council District')

In [21]:
council_district

Council District
19.0    3451
17.0    1488
21.0    1396
5.0     1333
6.0     1107
18.0     649
7.0      590
20.0     404
2.0      383
24.0     369
8.0      316
15.0     266
16.0     258
25.0     154
13.0     119
26.0     114
14.0      81
3.0       79
1.0       71
11.0      66
27.0      66
29.0      55
30.0      54
35.0      52
9.0       49
23.0      47
4.0       45
12.0      36
22.0      31
34.0      31
28.0      29
10.0      24
31.0      24
33.0      23
32.0      12
dtype: int64

In [22]:
zip = permits.value_counts('ZIP')
zip

ZIP
37203    2622
37206    1625
37208    1159
37212    1159
37207    1065
37209     971
37216     783
37201     581
37210     571
37204     458
37219     396
37211     395
37214     343
37205     184
37115     163
37217     139
37215     127
37218      87
37013      83
37221      71
37076      69
37138      60
37228      35
37072      29
37220      29
37189      26
37027      24
37080      11
37143       7
37064       2
37213       1
37135       1
37122       1
37015       1
0           1
dtype: int64

In [23]:
permit_subtype = permits.value_counts('Permit Subtype').to_frame().reset_index()
permit_subtype

,Permit Subtype,0
0,CAZ10A003,6651
1,CAZ10A001,4835
2,CAZ10A002,1793


In [24]:
#01 = Owner Occupied
#02 =  Non-owner occupied
#03 = Multifamily

In [25]:
permits_slice = permits[['Permit Subtype', 'Permit Status', 'Date Issued', 'Expiration Date']]
permits_slice.head()

,Permit Subtype,Permit Status,Date Issued,Expiration Date
0,CAZ10A003,EXPIRED,6/16/2016 0:00,6/16/2022 0:00
1,CAZ10A001,EXPIRED,7/21/2015 0:00,7/21/2022 0:00
2,CAZ10A001,ISSUED,6/26/2015 0:00,6/26/2023 0:00
3,CAZ10A003,EXPIRED,6/16/2016 0:00,6/16/2022 0:00
4,CAZ10A002,ISSUED,7/30/2015 0:00,7/30/2023 0:00


In [26]:
permits_sliced = permits_slice[['Permit Subtype', 'Permit Status']].value_counts()
permits_sliced

Permit Subtype  Permit Status
CAZ10A003       ISSUED           3481
CAZ10A001       EXPIRED          2457
CAZ10A003       EXPIRED          2245
CAZ10A001       ISSUED           1998
CAZ10A002       EXPIRED          1104
CAZ10A003       CANCELLED         725
CAZ10A002       ISSUED            474
CAZ10A001       CANCELLED         359
CAZ10A002       CANCELLED         212
CAZ10A003       PENDING           193
CAZ10A001       PENDING            10
                HOLD                5
CAZ10A003       HOLD                5
CAZ10A001       REVOKED             4
CAZ10A002       REVOKED             2
CAZ10A001       EXPIREEXT           2
CAZ10A003       REFUNDED            1
CAZ10A002       HOLD                1
CAZ10A003       REVOKED             1
dtype: int64

In [27]:
sliced_permits = permits_sliced.to_frame().reset_index()
sliced_permits

,Permit Subtype,Permit Status,0
0,CAZ10A003,ISSUED,3481
1,CAZ10A001,EXPIRED,2457
2,CAZ10A003,EXPIRED,2245
3,CAZ10A001,ISSUED,1998
4,CAZ10A002,EXPIRED,1104
5,CAZ10A003,CANCELLED,725
6,CAZ10A002,ISSUED,474
7,CAZ10A001,CANCELLED,359
8,CAZ10A002,CANCELLED,212
9,CAZ10A003,PENDING,193


In [28]:
sliced_permits.columns = ['Permit Subtype', 'Permit Status', 'Count of Permits']
sliced_permits

,Permit Subtype,Permit Status,Count of Permits
0,CAZ10A003,ISSUED,3481
1,CAZ10A001,EXPIRED,2457
2,CAZ10A003,EXPIRED,2245
3,CAZ10A001,ISSUED,1998
4,CAZ10A002,EXPIRED,1104
5,CAZ10A003,CANCELLED,725
6,CAZ10A002,ISSUED,474
7,CAZ10A001,CANCELLED,359
8,CAZ10A002,CANCELLED,212
9,CAZ10A003,PENDING,193


In [29]:
sliced_permits.sort_values('Permit Subtype')

,Permit Subtype,Permit Status,Count of Permits
1,CAZ10A001,EXPIRED,2457
3,CAZ10A001,ISSUED,1998
15,CAZ10A001,EXPIREEXT,2
7,CAZ10A001,CANCELLED,359
10,CAZ10A001,PENDING,10
11,CAZ10A001,HOLD,5
13,CAZ10A001,REVOKED,4
4,CAZ10A002,EXPIRED,1104
6,CAZ10A002,ISSUED,474
8,CAZ10A002,CANCELLED,212


In [30]:
#01 = Owner Occupied
#02 =  Non-owner occupied
#03 = Multifamily

In [40]:
permits_zip = permits[['ZIP', 'Permit Owner ZIP']]
permits_zip

,ZIP,Permit Owner ZIP
0,37201,37201
1,37216,37216
2,37206,37203
3,37201,37201
4,37206,37206
...,...,...
13274,37211,37211
13275,37203,94109
13276,37214,37214
13277,37203,33131


In [41]:
outside_nash = permits_zip[permits_zip['ZIP']!=permits_zip['Permit Owner ZIP']]
outside_nash

,ZIP,Permit Owner ZIP
2,37206,37203
5,37201,10017
6,37203,95630
11,37208,37067
12,37206,55369
...,...,...
13269,37210,M4P 1E4
13271,37203,38017
13272,37212,78041
13275,37203,94109


In [42]:
outside_nash.value_counts('Permit Owner ZIP')

Permit Owner ZIP
37215     585
33131     343
37205     338
37203     325
37204     273
         ... 
46227       1
46883       1
47553       1
48017       1
T5L3W9      1
Length: 837, dtype: int64

In [43]:
outside_nash = outside_nash.sort_values('Permit Owner ZIP').fillna(0)
outside_nash

,ZIP,Permit Owner ZIP
1704,37207,10001
4273,37216,10002
4893,37216,10002
5066,37212,10003
2614,37206,10008
...,...,...
12016,37217,0
12115,37207,0
12719,37216,0
12969,37212,0


In [47]:
zip_count = outside_nash.value_counts()
zip_count

ZIP    Permit Owner ZIP
37203  33131               343
       37215               216
       94111               164
37212  37204               119
37208  37215               115
                          ... 
37206  78722                 1
       78727                 1
       78746                 1
       78759                 1
37207  63936                 1
Length: 1505, dtype: int64

In [48]:
zip_count.to_frame()

0
ZIP   Permit Owner ZIP     
37203 33131             343
      37215             216
      94111             164
37212 37204             119
37208 37215             115
...                     ...
37206 78722               1
      78727               1
      78746               1
      78759               1
37207 63936               1

[1505 rows x 1 columns]

In [53]:
zip_count = zip_count.reset_index()

In [54]:
zip_count.columns = ['Airbnb Zip', 'Permit Owner Zip', 'Count of Permits']
zip_count

,Airbnb Zip,Permit Owner Zip,Count of Permits
0,37203,33131,343
1,37203,37215,216
2,37203,94111,164
3,37212,37204,119
4,37208,37215,115
...,...,...,...
1500,37206,78722,1
1501,37206,78727,1
1502,37206,78746,1
1503,37206,78759,1


In [57]:
zip_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1505 entries, 0 to 1504
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Airbnb Zip        1505 non-null   object
 1   Permit Owner Zip  1505 non-null   object
 2   Count of Permits  1505 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 35.4+ KB


In [58]:
zip_count['Count of Permits'] = zip_count['Count of Permits'].apply(str)
zip_count

,Airbnb Zip,Permit Owner Zip,Count of Permits
0,37203,33131,343
1,37203,37215,216
2,37203,94111,164
3,37212,37204,119
4,37208,37215,115
...,...,...,...
1500,37206,78722,1
1501,37206,78727,1
1502,37206,78746,1
1503,37206,78759,1


In [60]:
zip_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1505 entries, 0 to 1504
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Airbnb Zip        1505 non-null   object
 1   Permit Owner Zip  1505 non-null   object
 2   Count of Permits  1505 non-null   object
dtypes: object(3)
memory usage: 35.4+ KB


In [66]:
zip_count.sort_values('Count of Permits', ascending = False)

,Airbnb Zip,Permit Owner Zip,Count of Permits
8,37219,10022,91
120,37203,37206,9
118,37203,37130,9
117,37207,37015,9
121,37203,6905,9
...,...,...,...
894,37211,37207,1
895,37211,60616,1
896,37211,75204,1
897,37211,77515,1
